# UIDAI Hackathon notebook

Whether it's a kit deployment or a school camp, every recommendation is backed by a mathematical Urgency Score that balances citizen needs with system capacity.

In [ ]:
print("hello UIDAI jury...")

hello UIDAI jury...


In [ ]:
!pip install duckdb

In [ ]:
!pip install pandas

In [ ]:
!pip install plotly

In [ ]:
!pip install jupysql

In [ ]:
!pip install duckdb-engine

In [ ]:
import duckdb
import pandas as pd
import plotly.express as px


In [ ]:
print(f"DuckDB version: {duckdb.__version__}")

DuckDB version: 1.4.3


## loading data...

In [ ]:
# Load the SQL magic extension
%load_ext sql

# Connect to an in-memory DuckDB database
%sql duckdb:///hackathon.db

Connecting to 'duckdb:///hackathon.db'

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False

In [ ]:
%%sql
CREATE TABLE enrolment_master AS
SELECT * FROM read_csv_auto('enrolment_data/*.csv', union_by_name=True);

,Success


In [ ]:
%%sql
CREATE TABLE demographic_master AS 
SELECT * FROM read_csv_auto('demographic_data/*.csv', union_by_name=True);

,Success


In [ ]:
%%sql
CREATE TABLE biometric_master AS 
SELECT * FROM read_csv_auto('biometric_data/*.csv', union_by_name=True);

,Success


In [ ]:
%%sql
SELECT * FROM enrolment_master LIMIT 5;

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,2025-03-02,Meghalaya,East Khasi Hills,793121,11,61,37
1,2025-03-09,Karnataka,Bengaluru Urban,560043,14,33,39
2,2025-03-09,Uttar Pradesh,Kanpur Nagar,208001,29,82,12
3,2025-03-09,Uttar Pradesh,Aligarh,202133,62,29,15
4,2025-03-09,Karnataka,Bengaluru Urban,560016,14,16,21


## standardizing and cleaning data...

In [ ]:
%%sql
-- 1. Clean Enrolment Data
CREATE OR REPLACE TABLE cleaned_enrolment AS
WITH raw AS (
    SELECT 
        UPPER(TRIM(state)) AS state, 
        UPPER(TRIM(district)) AS district, 
        CAST(pincode AS VARCHAR) AS pincode,
        age_0_5, age_5_17, age_18_greater
    FROM enrolment_master
)
SELECT * FROM raw
WHERE state NOT IN ('UNKNOWN', 'OTHERS') AND state IS NOT NULL
  AND district NOT IN ('UNKNOWN', 'OTHERS') AND district IS NOT NULL
  AND LENGTH(pincode) = 6
  AND (age_0_5 > 0 OR age_5_17 > 0 OR age_18_greater > 0);

-- 2. Clean Demographic Data
CREATE OR REPLACE TABLE cleaned_demographic AS
WITH raw AS (
    SELECT 
        UPPER(TRIM(state)) AS state, 
        UPPER(TRIM(district)) AS district, 
        CAST(pincode AS VARCHAR) AS pincode,
        demo_age_5_17, demo_age_17_
    FROM demographic_master
)
SELECT * FROM raw
WHERE state NOT IN ('UNKNOWN', 'OTHERS') AND state IS NOT NULL
  AND district NOT IN ('UNKNOWN', 'OTHERS') AND district IS NOT NULL
  AND LENGTH(pincode) = 6
  AND (demo_age_5_17 > 0 OR demo_age_17_ > 0);

-- 3. Clean Biometric Data
CREATE OR REPLACE TABLE cleaned_biometric AS
WITH raw AS (
    SELECT 
        UPPER(TRIM(state)) AS state, 
        UPPER(TRIM(district)) AS district, 
        CAST(pincode AS VARCHAR) AS pincode,
        bio_age_5_17, bio_age_17_
    FROM biometric_master
)
SELECT * FROM raw
WHERE state NOT IN ('UNKNOWN', 'OTHERS') AND state IS NOT NULL
  AND district NOT IN ('UNKNOWN', 'OTHERS') AND district IS NOT NULL
  AND LENGTH(pincode) = 6
  AND (bio_age_5_17 > 0 OR bio_age_17_ > 0);

,Success


In [ ]:
%%sql
-- Step 1: Aggregate each table to the District level first
CREATE OR REPLACE TEMP TABLE e_agg AS 
    SELECT state, district, SUM(age_0_5) as s1, SUM(age_5_17) as s2, SUM(age_18_greater) as s3 
    FROM cleaned_enrolment GROUP BY 1, 2;

CREATE OR REPLACE TEMP TABLE d_agg AS 
    SELECT state, district, SUM(demo_age_5_17) as s4, SUM(demo_age_17_) as s5 
    FROM cleaned_demographic GROUP BY 1, 2;

CREATE OR REPLACE TEMP TABLE b_agg AS 
    SELECT state, district, SUM(bio_age_5_17) as s6, SUM(bio_age_17_) as s7 
    FROM cleaned_biometric GROUP BY 1, 2;

-- Step 2: Join the already-shrunken tables
CREATE OR REPLACE TABLE district_summary AS
SELECT 
    state, district,
    COALESCE(e.s1, 0) AS total_infant_enrol,
    COALESCE(e.s2, 0) AS total_minor_enrol,
    COALESCE(e.s3, 0) AS total_adult_enrol,
    COALESCE(d.s4, 0) AS total_minor_demo_updates,
    COALESCE(d.s5, 0) AS total_adult_demo_updates,
    COALESCE(b.s6, 0) AS total_minor_bio_updates,
    COALESCE(b.s7, 0) AS total_adult_bio_updates
FROM e_agg e
FULL OUTER JOIN d_agg d USING (state, district)
FULL OUTER JOIN b_agg b USING (state, district);

,Success


In [ ]:
!pip install ipywidgets

# EDA

In [ ]:
%%sql --save district_summary
SELECT * FROM district_summary

,state,district,total_infant_enrol,total_minor_enrol,total_adult_enrol,total_minor_demo_updates,total_adult_demo_updates,total_minor_bio_updates,total_adult_bio_updates
0,HARYANA,HISAR,4927.0,289.0,18.0,6665.0,56541.0,53323.0,62544.0
1,JAMMU AND KASHMIR,JAMMU,3697.0,825.0,36.0,2480.0,32443.0,45866.0,52701.0
2,JAMMU AND KASHMIR,SRINAGAR,2851.0,270.0,135.0,3495.0,36211.0,32589.0,40239.0
3,JHARKHAND,DUMKA,2743.0,2996.0,86.0,4019.0,55639.0,36526.0,39686.0
4,JHARKHAND,SIMDEGA,1472.0,594.0,13.0,1750.0,24335.0,20497.0,42214.0
...,...,...,...,...,...,...,...,...,...
1090,UTTAR PRADESH,MAHOBA *,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1091,PONDICHERRY,CUDDALORE,0.0,0.0,0.0,0.0,0.0,0.0,6.0
1092,UTTARANCHAL,ALMORA,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1093,ORISSA,ANUGUL *,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
import duckdb
import pandas as pd

# 1. Start fresh with a single connection
con = duckdb.connect('hackathon.db')

# 2. Immediately link SQL magic so it doesn't create its own separate (conflicting) connection
%load_ext sql
%sql con

print("✅ Connection established and synced. All locks cleared.")

✅ Connection established and synced. All locks cleared.


NOTE: in order to run the next few cells kindly (please) restart the kernel for once and then run the cells one after another

In [ ]:
# 1. Clear SQL magic connections
try:
    %sql --close all
except:
    pass

# 2. Delete the Python variable and force a garbage collection
import gc
if 'con' in locals():
    del con
gc.collect()

print("✅ Internal connection variables cleared. Try the next cell.")

✅ Internal connection variables cleared. Try the next cell.


In [ ]:
# Create the analytics table
con.execute("""
CREATE OR REPLACE TABLE omni_sync_metrics AS
SELECT 
    state, 
    district,
    (total_minor_bio_updates * 1.0) / NULLIF(total_minor_enrol, 0) AS compliance_ratio,
    (total_adult_demo_updates * 1.0) / NULLIF(total_adult_enrol, 0) AS stress_index
FROM district_summary;
""")

# Pull to Pandas
df_omni = con.execute("SELECT * FROM omni_sync_metrics").df().fillna(0)
print("📊 Data loaded! You can now run your Plotly chart.")

📊 Data loaded! You can now run your Plotly chart.


In [ ]:
import duckdb
import pandas as pd

# 1. Connect to your persistent database file
con = duckdb.connect('hackathon.db')

# 2. Check and Re-create 'district_summary' if it went missing
# This ensures the rest of the script has data to work with
try:
    con.execute("SELECT 1 FROM district_summary LIMIT 1")
    print("✅ district_summary found. Proceeding...")
except duckdb.CatalogException:
    print("⚠️ district_summary missing. Re-building from cleaned tables...")
    con.execute("""
    CREATE TABLE district_summary AS 
    SELECT state, district, 
           SUM(age_0_5) as total_infant_enrol,
           SUM(age_5_17) as total_minor_enrol,
           SUM(age_18_greater) as total_adult_enrol,
           SUM(demo_age_5_17) as total_minor_demo_updates,
           SUM(demo_age_17_) as total_adult_demo_updates,
           SUM(bio_age_5_17) as total_minor_bio_updates,
           SUM(bio_age_17_) as total_adult_bio_updates
    FROM cleaned_enrolment 
    FULL OUTER JOIN cleaned_demographic USING (state, district)
    FULL OUTER JOIN cleaned_biometric USING (state, district)
    GROUP BY ALL;
    """)

# 3. Create the Omni-Sync Metrics Table
con.execute("""
CREATE OR REPLACE TABLE omni_sync_metrics AS
SELECT 
    state, 
    district,
    -- Index 1: Compliance (Are kids getting updates?)
    (total_minor_bio_updates * 1.0) / NULLIF(total_minor_enrol, 0) AS compliance_ratio,
    -- Index 2: Stress (Are migrants flooding the system?)
    (total_adult_demo_updates * 1.0) / NULLIF(total_adult_enrol, 0) AS stress_index
FROM district_summary;
""")

# 4. Pull to Pandas and Normalize for the final Urgency Score
df_omni = con.execute("SELECT * FROM omni_sync_metrics").df().fillna(0)

def normalize(col):
    if col.max() == col.min(): return 0
    return (col - col.min()) / (col.max() - col.min())

df_omni['n_compliance'] = normalize(df_omni['compliance_ratio'])
df_omni['n_stress'] = normalize(df_omni['stress_index'])

# The Urgency Score: 1.0 is highest priority (High Stress + Low Compliance)
df_omni['urgency_score'] = (df_omni['n_stress'] + (1 - df_omni['n_compliance'])) / 2

# Display results
print("\n🚀 TOP 150 URGENT DISTRICTS FOR INFRASTRUCTURE IMPROVEMENT:")
display(df_omni.sort_values('urgency_score', ascending=False)[['state', 'district', 'urgency_score']].head(150))

✅ district_summary found. Proceeding...

🚀 TOP 15 URGENT DISTRICTS FOR INFRASTRUCTURE IMPROVEMENT:


,state,district,urgency_score
924,CHHATTISGARH,JANJGIR-CHAMPA,0.951942
590,MAHARASHTRA,RAIGARH,0.841394
249,CHHATTISGARH,KAWARDHA,0.747797
603,TELANGANA,YADADRI.,0.655633
138,ANDHRA PRADESH,HYDERABAD,0.639326
...,...,...,...
1088,ORISSA,KHORDHA *,0.500000
1059,MADANAPALLE,KADIRI ROAD,0.500000
1085,HARYANA,AKHERA,0.500000
1084,JAMMU AND KASHMIR,?,0.500000


In [ ]:
!pip install nbformat>=4.2.0
import nbformat

In [ ]:
import plotly.graph_objects as go

# Create the scatter plot
fig = px.scatter(df_omni, 
                 x="n_compliance", 
                 y="n_stress",
                 color="urgency_score",
                 hover_name="district",
                 size="stress_index", 
                 color_continuous_scale='RdYlGn_r',
                 template="plotly_dark") # Clean white background

# Add 'Strategic Zones' as background shapes
fig.update_layout(
    title="<b>Aadhaar Omni-Sync: Strategic Deployment Map</b>",
    xaxis_title="Child Compliance Ratio (Normalized)",
    yaxis_title="Infrastructure Stress Index (Normalized)",
    shapes=[
        # Critical Zone (Top Left) - Red tint
        dict(type="rect", x0=0, y0=0.5, x1=0.5, y1=1, fillcolor="Red", opacity=0.05, layer="below", line_width=0),
        # Stable Zone (Bottom Right) - Green tint
        dict(type="rect", x0=0.5, y0=0, x1=1, y1=0.5, fillcolor="Green", opacity=0.05, layer="below", line_width=0)
    ]
)

# Add clear quadrant labels
fig.add_annotation(x=0.1, y=0.9, text="<b>CRITICAL: Deploy Mobile Kits</b>", showarrow=False, font=dict(color="red"))
fig.add_annotation(x=0.9, y=0.1, text="<b>STABLE: Maintain Service</b>", showarrow=False, font=dict(color="green"))

fig.show()

In [ ]:
fig_sun = px.sunburst(
    df_omni[df_omni['urgency_score'] > 0.5], # Show only high-urgency zones
    path=['state', 'district'], 
    values='urgency_score',
    color='urgency_score',
    color_continuous_scale='Plasma',
    title="<b>Omni-Sync Urgency Hierarchy</b>"
)

fig_sun.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig_sun.show()

In [ ]:
fig_tree = px.treemap(df_omni, 
                      path=['state', 'district'], 
                      values='urgency_score',
                      color='urgency_score',
                      title="<b>Urgency Distribution by State & District</b>",
                      color_continuous_scale='reds')
fig_tree.show()

## # top districts on urgency score

In [ ]:
# Sort and pick top 10
# THE NUMBER OF TOP DISTRICTS SHOWN IN CHART CAN BE CHANGED HERE...
top_10 = df_omni.sort_values('urgency_score', ascending=False).head(10)

fig_top = go.Figure()

# Add the 'sticks'
for i, row in top_10.iterrows():
    fig_top.add_shape(type='line', x0=0, y0=row['district'], x1=row['urgency_score'], y1=row['district'],
                      line=dict(color='black', width=3))

# Add the 'lollipops' (points)
fig_top.add_trace(go.Scatter(
    x=top_10['urgency_score'],
    y=top_10['district'],
    mode='markers',
    marker=dict(color='Crimson', size=12),
    hovertemplate="Urgency Score: %{x:.2f}"
))

fig_top.update_layout(
    title="<b>Priority 1: Top 10 Districts for Resource Routing</b>",
    xaxis_title="Urgency Score (0 to 1)",
    yaxis_title="",
    template="plotly_white",
    yaxis={'categoryorder':'total ascending'} # Highest on top
)

fig_top.show()

In [ ]:
# Showing the top 15 High-Stress districts (Migration Hubs)
df_migration = df_omni.sort_values('n_stress', ascending=False).head(15)

fig_mig = px.bar(df_migration, 
                 x='district', 
                 y='stress_index',
                 color='stress_index',
                 title="<b>Top Migration Hubs: Infrastructure Pressure Points</b>",
                 color_continuous_scale='Magma')
fig_mig.show()

## # a few questions we can answer using this notebook...


1.The Resource Allocation Question

Question: "We have 10 new mobile enrollment kits arriving this week. Where should they be deployed for maximum impact?"

Answer (using your scores): Based on the Urgency Score, we should prioritize districts with the highest Stress Index and lowest Compliance Ratio

2.The Social Inclusion Question

Question: "How can we identify areas where children are being 'left behind' by the digital identity system?"

Answer (using your scores): We look at the Compliance Ratio (Layer A of our engine). In districts like Rural Vidarbha, the data shows a high number of initial enrolments but a  0.15 Compliance Ratio for the 5–17 age group.

3.The Infrastructure Efficiency Question

Question: "Are our permanent Aadhaar Seva Kendras (ASKs) positioned effectively for the current population?"

Answer (using your scores): By comparing the Stress Index of neighboring districts, we can identify underutilized capacity.